In [1]:
import pandas as pd

In [2]:
loyalty_scores = pd.read_excel("./data/grocery_database.xlsx", sheet_name="loyalty_scores")
customer_details = pd.read_excel("./data/grocery_database.xlsx", sheet_name="customer_details")
transactions = pd.read_excel("./data/grocery_database.xlsx", sheet_name="transactions")

In [3]:
loyalty_scores.head()

,customer_id,customer_loyalty_score
0,104,0.587
1,69,0.156
2,525,0.959
3,181,0.418
4,796,0.570


In [4]:
customer_details.head()

,customer_id,distance_from_store,gender,credit_score
0,74,3.38,F,0.59
1,524,4.76,F,0.52
2,607,4.45,F,0.49
3,343,0.91,M,0.54
4,322,3.02,F,0.63


In [5]:

transactions.head()

,customer_id,transaction_date,transaction_id,product_area_id,num_items,sales_cost
0,1,2020-04-10,435657533999,3,7,19.16
1,1,2020-04-10,435657533999,2,5,7.71
2,1,2020-06-02,436189770685,4,4,26.97
3,1,2020-06-02,436189770685,1,2,38.52
4,1,2020-06-10,436265380298,4,4,22.13


In [6]:
# Create the customer level dataset
data_for_regression = pd.merge(customer_details, loyalty_scores, how="left", on="customer_id")

sales_summary = transactions.groupby("customer_id").agg({
    "sales_cost": "sum",
    "num_items": "sum",
    "transaction_id": "count",
    "product_area_id": "nunique"
}).reset_index()


In [7]:
sales_summary.columns = ["customer_id", "total_sales", "total_items", "transaction_count", "product_area_count"]


In [8]:
sales_summary["average_basket_value"] = sales_summary["total_sales"] / sales_summary["transaction_count"]

In [9]:
data_for_regression = pd.merge(data_for_regression, sales_summary, how="inner", on="customer_id")

In [10]:
data_for_regression

,customer_id,distance_from_store,gender,credit_score,customer_loyalty_score,total_sales,total_items,transaction_count,product_area_count,average_basket_value
0,74,3.38,F,0.59,0.263,2563.71,297,44,5,58.266136
1,524,4.76,F,0.52,0.298,2996.02,357,49,5,61.143265
2,607,4.45,F,0.49,0.337,2853.82,350,49,5,58.241224
3,343,0.91,M,0.54,0.873,2388.31,272,54,5,44.227963
4,322,3.02,F,0.63,0.350,2401.64,278,50,5,48.032800
...,...,...,...,...,...,...,...,...,...,...
865,372,4.38,F,0.50,0.321,3423.01,338,49,5,69.857347
866,104,2.36,F,0.63,0.587,3648.08,280,49,5,74.450612
867,393,1.87,M,0.59,NaN,3067.83,254,53,5,57.883585
868,373,0.21,M,0.47,0.972,3303.80,264,51,5,64.780392


In [11]:
regression_modeling = data_for_regression.loc[data_for_regression["customer_loyalty_score"].notna()]
regression_scoring = data_for_regression.loc[data_for_regression["customer_loyalty_score"].isna()]

In [12]:
regression_scoring.drop(columns=['customer_loyalty_score'], axis=1, inplace=True)

C:\Users\abhir\AppData\Local\Temp\ipykernel_320\3924385571.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regression_scoring.drop(columns=['customer_loyalty_score'], axis=1, inplace=True)


In [13]:
import pickle

In [14]:
pickle.dump(regression_modeling, open("data/regression_modeling.p", "wb"))
pickle.dump(regression_scoring, open("data/abc_regression_scoring.p", "wb"))